# class 12: Intro, explainable ML

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
#import matplotlib as mpl
import matplotlib.pyplot as plt

#from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestRegressor

from sklearn import tree
from graphviz import Source
import graphviz
from IPython.display import Image  
from sklearn.tree import export_graphviz
# import pydotplus
#from sklearn.externals.six import StringIO  
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

## Partial dependence plots

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)
y = cal_housing.target

#y -= y.mean()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0
)

In [3]:
##
np.mean(y_train)
# rescale in a untypically way

2.0696849434754525

In [4]:
# the original data seem to be scaled to mean 0 and stedev 1, let us put them back on mare resanable scale
y_train2 = 750 + y_train*300

In [5]:
rf_house = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=100,max_features=2)
rf_house.fit(X_train, y_train2)  

RandomForestRegressor(max_depth=50, max_features=2, random_state=0)

In [6]:
X["prediction"] = rf_house.predict(X)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,prediction
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,2084.81745
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,1896.51036
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,1877.17224
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,1805.16603
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,1687.44906


In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence

print('Computing partial dependence plots...')
tic = time()
features = ['MedInc', 'AveOccup', 'HouseAge', 'AveRooms']
display = plot_partial_dependence(
       rf_house, X_train, features, grid_resolution=20
)
print(f"done in {time() - tic:.3f}s")
display.figure_.suptitle(
    'Partial dependence of house value on non-location features\n'
    'for the California housing dataset, with MLPRegressor'
)
display.figure_.subplots_adjust(hspace=0.3)

## SHAP values

## Titanic 

In [ ]:
df = pd.read_csv('../data/TitanicTrain.csv') # Load the data
one_hot_data = pd.get_dummies(df[["Sex", "Pclass", "Age", "PassengerId","Survived"]])
one_hot_data=one_hot_data[["Sex_male", "Pclass", "Age", "PassengerId","Survived"]].dropna()
Y = one_hot_data['Survived']
X =  one_hot_data[['Age', 'Pclass','Sex_male', 'PassengerId']]
# Split the data into train and test data:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
rf = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=100,max_features=2)
rf.fit(X_train, Y_train)  
print(rf.feature_importances_)
importances = rf.feature_importances_
indices = np.argsort(importances)
features = X_train.columns
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train)

In [ ]:
shap_values.shape

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

To understand how a single feature effects the output of the model we can plot the SHAP value of that feature vs. the value of the feature for all the examples in a dataset. Since SHAP values represent a feature's responsibility for a change in the model output, the plot below represents the change in predicted survival prob. as `Age` changes. Vertical dispersion at a single value of age represents interaction effects with other features. To help reveal these interactions dependence_plot automatically selects another feature for coloring.

In [ ]:
shap.dependence_plot("Age", shap_values, X_train)

### Force plots

The plot below shows features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue 

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
# load JS visualization code to notebook
shap.initjs()

shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

If we take many explanations such as the one shown above, rotate them 90 degrees, and then stack them horizontally, we can see explanations for an entire dataset (in the notebook this plot is interactive):

In [ ]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, X_train)

### Tasks

1. Visualize SHAP values for the Boston data 
2. Look at interactions